In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd gdrive/My\ Drive

/content/gdrive/My Drive


In [3]:
# Install necessary packages -> uncomment what is currently needed

!pip install unidecode
!pip install contractions
!pip install wordsegment
!pip install -U symspellpy
!pip install emoji --upgrade
!pip install -U imbalanced-learn
!pip install bert-for-tf2
!pip install transformers
# !pip install nltk

Requirement already up-to-date: symspellpy in /usr/local/lib/python3.6/dist-packages (6.5.2)
Requirement already up-to-date: emoji in /usr/local/lib/python3.6/dist-packages (0.5.4)
Requirement already up-to-date: imbalanced-learn in /usr/local/lib/python3.6/dist-packages (0.6.2)


In [4]:
# All imports - DO NOT CHANGE THE ORDER OF INSTRUCTIONS
!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo

import re
import os
import sys
import json

if not 'bert_repo' in sys.path:
    sys.path.insert(0, 'bert_repo')

import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.model_selection import train_test_split
import spacy
from bs4 import BeautifulSoup
import unidecode
import contractions
import gensim.downloader as api
import re
import wordsegment
import pkg_resources
from symspellpy.symspellpy import SymSpell, Verbosity
import emoji
from imblearn.over_sampling import SMOTE
import tensorflow as tf2
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from modeling import BertModel, BertConfig
from tokenization import FullTokenizer, convert_to_unicode
from extract_features import InputExample, convert_examples_to_features
from tqdm import tqdm
#import tensorflow_addons as tfa
# import nltk
from google.colab import auth, drive
# nltk.download('punkt')

wordsegment.load()

# Load SymSpell -> package for correcting misspellings
sym_spell = SymSpell(2, 7)

dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt")

sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

# get TF logger 
log = logging.getLogger('tensorflow')
log.handlers = []

Using TensorFlow backend.


In [5]:
#Import data
training_examples_url = 'https://raw.githubusercontent.com/piotrjaniszewski1/SemEval-2020-Task12/master/data2019/olid-training-v1.0.tsv'
testing_examples_A_url = 'https://raw.githubusercontent.com/piotrjaniszewski1/Offensive-Language-Identification-and-Categorization/master/data2019/testset-levela.tsv'
testing_labels_A_url = 'https://raw.githubusercontent.com/piotrjaniszewski1/Offensive-Language-Identification-and-Categorization/master/data2019/labels-levela.csv'

training_dataset = pd.read_csv(training_examples_url, delimiter='\t')
testing_dataset_examples_A = pd.read_csv(testing_examples_A_url, delimiter='\t')
testing_dataset_labels_A = pd.read_csv(testing_labels_A_url, delimiter=',')

print(training_dataset.head())

      id                                              tweet  ... subtask_b subtask_c
0  86426  @USER She should ask a few native Americans wh...  ...       UNT       NaN
1  90194  @USER @USER Go home you’re drunk!!! @USER #MAG...  ...       TIN       IND
2  16820  Amazon is investigating Chinese employees who ...  ...       NaN       NaN
3  62688  @USER Someone should'veTaken" this piece of sh...  ...       UNT       NaN
4  43605  @USER @USER Obama wanted liberals &amp; illega...  ...       NaN       NaN

[5 rows x 5 columns]


In [0]:
#Import 2020 data

training_dataset2020 = pd.read_csv('Colab Notebooks/task_a_distant.tsv', delimiter='\t', nrows=100000)

In [7]:
def convert_averages(row):
    return 'OFF' if row['average'] >= 0.4 else 'NOT'


training_dataset2020 = training_dataset2020.drop(labels=['id', 'std'], axis=1)
training_dataset2020['average'] = training_dataset2020.apply(lambda row: convert_averages(row), axis=1)
training_dataset2020 = training_dataset2020.rename(columns={ 'average': 'subtask_a', 'text': 'tweet' })
training_dataset = training_dataset.append(training_dataset2020)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


## Additional datasets

In [0]:
# Additional datasets
set_urls = [
            'https://raw.githubusercontent.com/piotrjaniszewski1/Offensive-Language-Identification-and-Categorization/master/Additional%20datasets/set1.csv',
            'https://raw.githubusercontent.com/piotrjaniszewski1/Offensive-Language-Identification-and-Categorization/master/Additional%20datasets/set2.csv',
            'https://raw.githubusercontent.com/piotrjaniszewski1/Offensive-Language-Identification-and-Categorization/master/Additional%20datasets/set3.csv',
            'https://raw.githubusercontent.com/piotrjaniszewski1/Offensive-Language-Identification-and-Categorization/master/Additional%20datasets/set4.csv',
            'https://raw.githubusercontent.com/piotrjaniszewski1/Offensive-Language-Identification-and-Categorization/master/Additional%20datasets/set5.csv',
]

sets = [pd.read_csv(url) for url in set_urls]

In [0]:
def labels_set1(row):
  return 'OFF' if 1 in [row['toxic'], row['severe_toxic'], row['obscene'], row['threat'], row['insult'], row['identity_hate']] else 'NOT'

def labels_set2(row):
  return 'OFF' if row['class'] != 2 else 'NOT'

def labels_set_ordinary(row):
  return 'OFF' if row['subtask_a'] == 1 else 'NOT'

In [10]:
'''
# Preprocess additional datasets

sets[0] = sets[0].rename(columns={ 'label': 'subtask_a' })
sets[0]['subtask_a'] = sets[0].apply(lambda row: labels_set_ordinary(row), axis=1)
sets[0] = sets[0].drop('id', axis=1)

sets[1] = sets[1].rename(columns={ 'comment_text': 'tweet', 'label': 'subtask_a' })
sets[1]['subtask_a'] = sets[1].apply(lambda row: labels_set1(row), axis=1)
sets[1] = sets[1].drop(labels=['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1)

sets[2] = sets[2].rename(columns={ 'label': 'subtask_a' })
sets[2]['subtask_a'] = sets[2].apply(lambda row: labels_set2(row), axis=1)
sets[2] = sets[2].drop(labels=['count', 'hate_speech', 'offensive_language', 'neither', 'class', 'Unnamed: 0'], axis=1)

sets[3] = sets[3].rename(columns={ 'Insult': 'subtask_a', 'Comment': 'tweet' })
sets[3]['subtask_a'] = sets[3].apply(lambda row: labels_set_ordinary(row), axis=1)
sets[3] = sets[3].drop('Date', axis=1)

sets[4] = sets[4].rename(columns={ 'Insult': 'subtask_a', 'Comment': 'tweet' })
sets[4]['subtask_a'] = sets[4].apply(lambda row: labels_set_ordinary(row), axis=1)
sets[4] = sets[4].drop(labels=['Date', 'Usage'], axis=1)

sets[5] = sets[5].rename(columns={ 'Insult': 'subtask_a', 'Comment': 'tweet' })
sets[5]['subtask_a'] = sets[5].apply(lambda row: labels_set_ordinary(row), axis=1)
sets[5] = sets[5].drop(labels=['Date', 'id', 'Usage'], axis=1)

sets_hate_only = [s[s['subtask_a'] == 'OFF'] for s in sets]

for s in sets: # one can change to sets_hate_only
  training_dataset = training_dataset.append(s)

sets_hate_only
'''

"\n# Preprocess additional datasets\n\nsets[0] = sets[0].rename(columns={ 'label': 'subtask_a' })\nsets[0]['subtask_a'] = sets[0].apply(lambda row: labels_set_ordinary(row), axis=1)\nsets[0] = sets[0].drop('id', axis=1)\n\nsets[1] = sets[1].rename(columns={ 'comment_text': 'tweet', 'label': 'subtask_a' })\nsets[1]['subtask_a'] = sets[1].apply(lambda row: labels_set1(row), axis=1)\nsets[1] = sets[1].drop(labels=['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1)\n\nsets[2] = sets[2].rename(columns={ 'label': 'subtask_a' })\nsets[2]['subtask_a'] = sets[2].apply(lambda row: labels_set2(row), axis=1)\nsets[2] = sets[2].drop(labels=['count', 'hate_speech', 'offensive_language', 'neither', 'class', 'Unnamed: 0'], axis=1)\n\nsets[3] = sets[3].rename(columns={ 'Insult': 'subtask_a', 'Comment': 'tweet' })\nsets[3]['subtask_a'] = sets[3].apply(lambda row: labels_set_ordinary(row), axis=1)\nsets[3] = sets[3].drop('Date', axis=1)\n\nsets[4] = sets[4].rename(col

# **Training and validation sets**

In [0]:
seed = 13

# prepare training examples
training_examples_A = training_dataset['tweet'][training_dataset['subtask_a'].notnull()]
training_examples_B = training_dataset['tweet'][training_dataset['subtask_b'].notnull()]
training_examples_C = training_dataset['tweet'][training_dataset['subtask_c'].notnull()]

# prepare test examples and labels
test_examples_A = testing_dataset_examples_A['tweet'][testing_dataset_examples_A['tweet'].notnull()]
test_labels_A = (testing_dataset_labels_A['label'][testing_dataset_labels_A['label'].notnull()] == 'OFF').astype(int)

# prepare training labels
training_labels_A = (training_dataset['subtask_a'][training_dataset['subtask_a'].notnull()] == 'OFF').astype(int)
training_labels_B = (training_dataset['subtask_b'][training_dataset['subtask_b'].notnull()] == 'TIN').astype(int)
c_mapping = {'IND': 0, 'GRP': 1, 'OTH': 2}
training_labels_C = training_dataset['subtask_c'][training_dataset['subtask_c'].notnull()].replace(c_mapping)

# split training set into training and validation
training_examples_A, validation_examples_A, training_labels_A, validation_labels_A = train_test_split(
    training_examples_A, training_labels_A, test_size=0.1, stratify=training_labels_A, random_state=seed)
training_examples_B, validation_examples_B, training_labels_B, validation_labels_B = train_test_split(
    training_examples_B, training_labels_B, test_size=0.1, stratify=training_labels_B, random_state=seed)
training_examples_C, validation_examples_C, training_labels_C, validation_labels_C = train_test_split(
    training_examples_C, training_labels_C, test_size=0.1, stratify=training_labels_C, random_state=seed)

training_x = np.array(training_examples_A)
validation_x = np.array(validation_examples_A)
training_y = np.array(training_labels_A)
validation_y = np.array(validation_labels_A)
test_x = np.array(test_examples_A)
test_y = np.array(test_labels_A)

# **Preprocessing**

### Common preprocessing functions

In [0]:
# remove html tags if exist
def strip_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    stripped_text = soup.get_text(separator=' ')
    return stripped_text


# remove unnecessary whitespaces
def remove_whitespace(text):
    text = text.strip()
    return ' '.join(text.split())


# remove accented chars (e.g. caffè -> caffe)
def remove_accented_chars(text):
    text = unidecode.unidecode(text)
    return text


# remove hashes and split words (e.g. '#fortTrump' -> 'fort trump')
def split_hashtags(text):
    splitted = text.split()
    new_word_sequence = []

    for chunk in splitted:
        if chunk[0] == '#':
            chunk = chunk[1:]
            new_word_sequence.extend(wordsegment.segment(chunk))
        else:
            new_word_sequence.append(chunk)
        
    return ' '.join(tuple(new_word_sequence))


def substitute_emojis(text):
    demojized_text = emoji.demojize(text)
    return re.compile('[_:]+').sub(' ', demojized_text)


def preprocess_common(text):
    text = strip_html_tags(text)
    text = contractions.fix(text)
    text = split_hashtags(text)
    text = substitute_emojis(text)
    text = remove_whitespace(text)
    text = remove_accented_chars(text)
    return text.lower()

In [0]:
# Remove redundant @user tokens
def remove_redundant_users(example):
    user_count = 0
    new_example = example[:]
    for i, token in reversed(list(enumerate(example))):
        if token == '@user':
            user_count += 1
        if user_count > 3:
            new_example.pop(i)
    else:
        user_count = 0

    return new_example

### Spacy preprocessing

In [0]:
# Try leaving '?' and '!' as far as punctuation is concerned

nlp = spacy.load('en_core_web_sm')

# exclude negation words from spacy stopwords list
deselect_stop_words = ['no', 'not', 'noone', 'none', 'lacks', 'lack', 'nor', 'never', 'neighter', 'hardly', 'nobody', 'nothing', 'lacking', 'nowhere']
for w in deselect_stop_words:
    nlp.vocab[w].is_stop = False

def preprocess_spacy(text):
    doc = nlp(text)

    clean_text = []
    
    for token in doc:
        flag = True
        edit = token.text
        '''
        # remove punctuations
        if token.pos_ == 'PUNCT' and flag == True and token.text != '@user': 
            flag = False
       
        # remove special characters
        if token.pos_ == 'SYM' and flag == True: 
            flag = False
        
        # remove numbers
        if (token.pos_ == 'NUM' or token.text.isnumeric()) and flag == True:
            flag = False

        # correct misspelings
        # if flag == True:
            # suggestions = sym_spell.lookup(edit, Verbosity.TOP, 2)
            # if len(suggestions) > 0:
                # edit = suggestions[0].term

        # remove stop words
        if token.is_stop and token.pos_ != 'NUM': 
            flag = False

        # convert tokens to base form
        elif token.lemma_ != '-PRON-' and flag == True:
            edit = token.lemma_
        '''
        # append tokens edited and not removed to list 
        if edit != '' and flag == True:
            clean_text.append(edit)        
    
    return clean_text

### Preprocessing execution

In [15]:
cleaned_x = [preprocess_spacy(example) for example in training_x[0:30]]
reduced_users_x = [remove_redundant_users(example) for example in cleaned_x]
print(reduced_users_x[0:30])
print(training_x[0:30])

[['@USER', 'This', 'one', 'is', 'just', 'cursed', 'like', 'this', '😟', 'why', 'not', 'report', 'her', 'to', 'the', 'police', '.', '🙄', '🤐', 'if', 'this', 'is', 'her', 'mum', 'she', 'is', 'very', 'very', 'very', 'stupid'], ['@USER', '@USER', '@USER', 'The', 'historic', 'moniker', 'for', 'the', 'Tories', 'is', 'The', 'Stupid', 'Party', '.', '  ', 'American', 'conservatives', 'test', 'out', 'with', 'lower', 'average', 'intelligence', 'than', 'liberals', '.'], ['Second', 'choice', 'always', 'wins', '🙂', '🙂', 'I', '’m', 'betting', 'I', '’ll', 'feel', 'headache', 'and', 'dizziness', 'as', 'soon', 'as', 'it', 'becomes', '5:30', 'am'], ['@USER', '@USER', 'Fuck', '@USER', 'and', 'everything', 'he', '’s', 'about', ',', 'including', '@USER', ',', 'Criminal', 'News', 'Network'], ['Life', '’s', 'shit', ',', 'the', 'earth', 'is', 'shit', 'and', 'everything', 'is', 'shit', 'unless', 'you', '’re', 'rich', '.', 'Wo', 'n’t', 'be', 'told', 'otherwise', 'because', 'it', '’s', 'strictly', 'facts'], ['@USER

# **BERT**

### BERT Preprocessing - single example


In [0]:
def preprocess_bert_before_tokenization(text):
    preprocessed_text = preprocess_common(text)
    spacy_x = preprocess_spacy(text)
    cleaned_x = remove_redundant_users(spacy_x)
    return ' '.join(cleaned_x)

### Getting the pre-trained model

In [17]:
!wget https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
!unzip wwm_uncased_L-24_H-1024_A-16.zip

--2020-03-04 11:18:15--  https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.128, 2404:6800:4008:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1248381879 (1.2G) [application/zip]
Saving to: ‘wwm_uncased_L-24_H-1024_A-16.zip.3’

wwm_uncased_L-24_H- 100%[===================>]   1.16G  84.5MB/s    in 16s     

2020-03-04 11:18:32 (75.3 MB/s) - ‘wwm_uncased_L-24_H-1024_A-16.zip.3’ saved [1248381879/1248381879]

Archive:  wwm_uncased_L-24_H-1024_A-16.zip
replace wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.meta? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  A

  inflating: wwm_uncased_L-24_H-1024_A-16/vocab.txt  
  inflating: wwm_uncased_L

### Building a tf.Module

In [0]:
def build_module_fn(config_path, vocab_path, do_lower_case=True):

    def bert_module_fn(is_training):
        """Spec function for a token embedding module."""

        input_ids = tf.placeholder(shape=[None, None], dtype=tf.int32, name="input_ids")
        input_mask = tf.placeholder(shape=[None, None], dtype=tf.int32, name="input_mask")
        token_type = tf.placeholder(shape=[None, None], dtype=tf.int32, name="segment_ids")

        config = BertConfig.from_json_file(config_path)
        model = BertModel(config=config, is_training=is_training,
                          input_ids=input_ids, input_mask=input_mask, token_type_ids=token_type)
          
        seq_output = model.all_encoder_layers[-1]
        pool_output = model.get_pooled_output()

        config_file = tf.constant(value=config_path, dtype=tf.string, name="config_file")
        vocab_file = tf.constant(value=vocab_path, dtype=tf.string, name="vocab_file")
        lower_case = tf.constant(do_lower_case)

        tf.add_to_collection(tf.GraphKeys.ASSET_FILEPATHS, config_file)
        tf.add_to_collection(tf.GraphKeys.ASSET_FILEPATHS, vocab_file)
            
        input_map = {"input_ids": input_ids,
                     "input_mask": input_mask,
                     "segment_ids": token_type}
        
        output_map = {"pooled_output": pool_output,
                      "sequence_output": seq_output}

        output_info_map = {"vocab_file": vocab_file,
                           "do_lower_case": lower_case}
                
        hub.add_signature(name="tokens", inputs=input_map, outputs=output_map)
        hub.add_signature(name="tokenization_info", inputs={}, outputs=output_info_map)

    return bert_module_fn

### Exporting the module

In [0]:
MODEL_DIR = "wwm_uncased_L-24_H-1024_A-16"

config_path = "/content/gdrive/My Drive/{}/bert_config.json".format(MODEL_DIR)
vocab_path = "/content/gdrive/My Drive/{}/vocab.txt".format(MODEL_DIR)

tags_and_args = []
for is_training in (True, False):
  tags = set()
  if is_training:
    tags.add("train")
  tags_and_args.append((tags, dict(is_training=is_training)))

module_fn = build_module_fn(config_path, vocab_path)
spec = hub.create_module_spec(module_fn, tags_and_args=tags_and_args)
spec.export("bert-module", 
            checkpoint_path="/content/gdrive/My Drive/{}/bert_model.ckpt".format(MODEL_DIR))


### Building the text preprocessing pipeline

In [0]:
def read_examples(str_list):
    """Read a list of `InputExample`s from a list of strings."""
    unique_id = 0
    for s in str_list:
        line = convert_to_unicode(s)
        if not line:
            continue
        
        text_a = line.strip()
        splitted = re.split(r'[.?!]\s*', text_a)
        text_b = splitted[-1]
        text_a.replace(text_b, '')


        yield InputExample(unique_id=unique_id, text_a=text_a, text_b=text_b)
        unique_id += 1

In [0]:
def features_to_arrays(features):

    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []

    for feature in features:
        all_input_ids.append(feature.input_ids)
        all_input_mask.append(feature.input_mask)
        all_segment_ids.append(feature.input_type_ids)

    return (np.array(all_input_ids, dtype='int32'), 
            np.array(all_input_mask, dtype='int32'), 
            np.array(all_segment_ids, dtype='int32'))

In [0]:
def build_preprocessor(voc_path, seq_len, lower=True):
    tokenizer = FullTokenizer(vocab_file=voc_path, do_lower_case=lower)

    def strings_to_arrays(sents):

        sents = np.atleast_1d(sents).reshape((-1,))

        examples = []
        for example in read_examples(sents):
            example.text_a = preprocess_bert_before_tokenization(example.text_a)
            examples.append(example)

        features = convert_examples_to_features(examples, seq_len, tokenizer)
        arrays = features_to_arrays(features)
        
        return arrays

    return strings_to_arrays

### Implementing a BERT Keras layer

In [0]:
class BertLayer(tf.keras.layers.Layer):
    def __init__(self, bert_path, seq_len=64, n_tune_layers=3, 
                 pooling="cls", do_preprocessing=True, verbose=False,
                 tune_embeddings=False, trainable=True, **kwargs):

        self.trainable = trainable
        self.n_tune_layers = n_tune_layers
        self.tune_embeddings = tune_embeddings
        self.do_preprocessing = do_preprocessing

        self.verbose = verbose
        self.seq_len = seq_len
        self.pooling = pooling
        self.bert_path = bert_path

        self.var_per_encoder = 16
        if self.pooling not in ["cls", "mean", "lstm", None]:
            raise NameError(
                f"Undefined pooling type (must be either 'cls', 'mean', or None, but is {self.pooling}"
            )

        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):

        self.bert = hub.Module(self.build_abspath(self.bert_path), 
                               trainable=self.trainable, name=f"{self.name}_module")

        trainable_layers = []
        if self.tune_embeddings:
            trainable_layers.append("embeddings")

        if self.pooling == "cls":
            trainable_layers.append("pooler")

        if self.n_tune_layers > 0:
            encoder_var_names = [var.name for var in self.bert.variables if 'encoder' in var.name]
            n_encoder_layers = int(len(encoder_var_names) / self.var_per_encoder)
            for i in range(self.n_tune_layers):
                trainable_layers.append(f"encoder/layer_{str(n_encoder_layers - 1 - i)}/")
        
        # Add module variables to layer's trainable weights
        for var in self.bert.variables:
            if any([l in var.name for l in trainable_layers]):
                self._trainable_weights.append(var)
            else:
                self._non_trainable_weights.append(var)

        if self.verbose:
            print("*** TRAINABLE VARS *** ")
            for var in self._trainable_weights:
                print(var)

        self.build_preprocessor()
        self.initialize_module()

        super(BertLayer, self).build(input_shape)

    def build_abspath(self, path):
        if path.startswith("https://") or path.startswith("gs://"):
          return path
        else:
          return os.path.abspath(path)

    def build_preprocessor(self):
        sess = tf.keras.backend.get_session()
        tokenization_info = self.bert(signature="tokenization_info", as_dict=True)
        vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                              tokenization_info["do_lower_case"]])
        self.preprocessor = build_preprocessor(vocab_file, self.seq_len, do_lower_case)

    def initialize_module(self):
        sess = tf.keras.backend.get_session()
        
        vars_initialized = sess.run([tf.is_variable_initialized(var) 
                                     for var in self.bert.variables])

        uninitialized = []
        for var, is_initialized in zip(self.bert.variables, vars_initialized):
            if not is_initialized:
                uninitialized.append(var)

        if len(uninitialized):
            sess.run(tf.variables_initializer(uninitialized))

    def call(self, input):
        if self.do_preprocessing:
          input = tf.numpy_function(self.preprocessor, 
                                    [input], [tf.int32, tf.int32, tf.int32], 
                                    name='preprocessor')
          for feature in input:
            feature.set_shape((None, self.seq_len))
        
        input_ids, input_mask, segment_ids = input
        
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        output = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)

        if self.pooling == "cls":
            pooled = output["pooled_output"]
        else:
            result = output["sequence_output"]

            if self.pooling == "lstm":
              lstm = tf.keras.layers.LSTM(1)
              pooled = lstm(result)

            else:
              input_mask = tf.cast(input_mask, tf.float32)
              mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)
              masked_reduce_mean = lambda x, m: tf.reduce_sum(mul_mask(x, m), axis=1) / (
                      tf.reduce_sum(m, axis=1, keepdims=True) + 1e-10)
              
              if self.pooling == "mean":
                pooled = masked_reduce_mean(result, input_mask)
              else:
                pooled = mul_mask(result, input_mask)

        return pooled

    def get_config(self):
        config_dict = {
            "bert_path": self.bert_path, 
            "seq_len": self.seq_len,
            "pooling": self.pooling,
            "n_tune_layers": self.n_tune_layers,
            "tune_embeddings": self.tune_embeddings,
            "do_preprocessing": self.do_preprocessing,
            "verbose": self.verbose
        }
        super(BertLayer, self).get_config()
        return config_dict

### Classification

In [22]:
inp = tf.keras.Input(shape=(1,), dtype=tf.string)
encoder = BertLayer(bert_path="./bert-module/", seq_len=200, tune_embeddings=False,
                    pooling="cls", n_tune_layers=24, verbose=False)

l1 = tf.keras.layers.Dense(768, activation='relu')
l2 = tf.keras.layers.Dense(768, activation='relu')
l3 = tf.keras.layers.Dense(768, activation='relu')
d = tf.keras.layers.Dropout(0.5)

pred = tf.keras.layers.Dense(1, activation='sigmoid')(encoder(inp))
model = tf.keras.models.Model(inputs=[inp], outputs=[pred])

From bert_repo/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
import keras.backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall + tf.keras.backend.epsilon()))

In [0]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                            np.unique(training_y),
                                            training_y)
class_weights /= max(class_weights)

# print(class_weights) # [majority class weight, minority class weight]

minority_count = len(training_labels_A[training_labels_A == 1])
majority_count = len(training_labels_A[training_labels_A == 0])

def weighted_loss(actual, predicted):
    weights = class_weights
    # weights = [0.0714, 1.0]
    bce = tf.keras.losses.BinaryCrossentropy()
    loss = bce(actual, predicted)
   
    return tf.keras.backend.mean(loss * weights)

In [25]:
model.summary()

model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=7e-6, ),
      loss=weighted_loss,
      metrics=[f1_m])

From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
bert_layer (BertLayer)       (None, 1024)              335141888 
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1025      
Total params: 335,142,913
Trainable params: 303,360,001
Non-trainable params: 31,782,912
_________________________________________________________________


### Training

In [0]:
import logging
logging.getLogger("tensorflow").setLevel(logging.WARNING)

In [0]:
saver = keras.callbacks.ModelCheckpoint("bert_large_H8_S256_B32.hdf5")

history = model.fit(training_x, training_y, validation_data=[validation_x, validation_y], batch_size=16, epochs=1, callbacks=[saver])

Train on 101916 samples, validate on 11324 samples


From bert_repo/extract_features.py:283: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.



  9568/101916 [=>............................] - ETA: 2:04:39 - loss: 0.1838 - f1_m: 0.7020

### Loss function graph

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Predict

In [0]:
predicted = model.predict(test_x, batch_size=16)

model_emb = tf.keras.models.Model(inputs=[model.layers[0].input], outputs=[model.layers[-3].output])
data = model_emb.predict(training_x)
np.savez('bert-embeddings.mpz', data3)

### Count f1-score

In [0]:
from sklearn.metrics import f1_score

def round_int(array, threshold):
  return np.array([np.ceil(x) if x >= threshold else np.floor(x) for x in array]).astype(np.float64)

flat_predicted = round_int(predicted[:, 0], 0.5)
print(f1_score(test_y, flat_predicted))

## Saving and restoring

In [0]:
import json
json.dump(model.to_json(), open("bert_H4_S256_B64.json", "w"))

In [0]:
model = tf.keras.models.model_from_json(json.load(open("bert_H4_S256_B64.json")), 
                                        custom_objects={"BertLayer": BertLayer})

model.load_weights("bert_H5_S256_B64.hdf5")

In [0]:
model.predict(training_x[:10])

### Freeze model

In [0]:
from tensorflow.python.framework.graph_util import convert_variables_to_constants
from tensorflow.python.tools.optimize_for_inference_lib import optimize_for_inference

def freeze_keras_model(model, export_path=None, clear_devices=True):
    """
    Freezes a Keras model into a pruned computation graph.

    @param model The Keras model to be freezed.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    
    sess = tf.keras.backend.get_session()
    graph = sess.graph
    
    with graph.as_default():

        input_tensors = model.inputs
        output_tensors = model.outputs
        dtypes = [t.dtype.as_datatype_enum for t in input_tensors]
        input_ops = [t.name.rsplit(":", maxsplit=1)[0] for t in input_tensors]
        output_ops = [t.name.rsplit(":", maxsplit=1)[0] for t in output_tensors]
        
        tmp_g = graph.as_graph_def()
        if clear_devices:
            for node in tmp_g.node:
                node.device = ""
        
        tmp_g = optimize_for_inference(
            tmp_g, input_ops, output_ops, dtypes, False)
        
        tmp_g = convert_variables_to_constants(sess, tmp_g, output_ops)
        
        if export_path is not None:
            with tf.gfile.GFile(export_path, "wb") as f:
                f.write(tmp_g.SerializeToString())
        
        return tmp_g

In [0]:
frozen_graph = freeze_keras_model(model, export_path="frozen_graph_4_256_64.pb")

In [0]:
!git clone https://github.com/gaphex/bert_experimental/

import tensorflow as tf
import numpy as np
import sys

sys.path.insert(0, "/content/bert_experimental")

from bert_experimental.finetuning.text_preprocessing import build_preprocessor
from bert_experimental.finetuning.graph_ops import load_graph

In [0]:
restored_graph = load_graph("frozen_graph_4_256_64.pb")

In [0]:
graph_ops = restored_graph.get_operations()
input_op, output_op = graph_ops[0].name, graph_ops[-1].name
print(input_op, output_op)

In [0]:
x = restored_graph.get_tensor_by_name(input_op + ':0')
y = restored_graph.get_tensor_by_name(output_op + ':0')

In [0]:
preprocessor = build_preprocessor("./uncased_L-12_H-768_A-12/vocab.txt", 64)
py_func = tf.numpy_function(preprocessor, [x], [tf.int32, tf.int32, tf.int32], name='preprocessor')

In [0]:
py_func = tf.numpy_function(preprocessor, [x], [tf.int32, tf.int32, tf.int32])

In [0]:
sess = tf.Session(graph=restored_graph)

In [0]:
y_out = sess.run(y, feed_dict={
        x: training_x[:10].reshape((-1,1))
    })

y_out